In [2]:
from pathlib import Path

import pandas as pd
import polars as ps
import seaborn as sb

In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Data Preprocessing

## Load Datasets

In [4]:
type_map: list = [ps.Utf8, ps.Utf8, ps.Int32, ps.Utf8, ps.Utf8, ps.Float32, ps.Float32, 
                  ps.Float32, ps.Float32, ps.Float32, ps.Float32, ps.Float32, ps.Float32, 
                  ps.Float32, ps.Float32, ps.Float32, ps.Float32, ps.Float32, ps.Int32,
                  ps.Datetime, ps.Float32, ps.Float32, ps.Float32, ps.Int32]
null_values: list = ['', '-']

In [5]:
dataset_type: str = 'train'
datasets: list = [
	ps.read_csv(file, encoding='utf8', dtypes=type_map, null_values=null_values, n_threads=8, use_pyarrow=True)
	for file in Path('data/{}'.format(dataset_type)).glob('*.csv')]
dataset: ps.DataFrame = ps.concat(datasets)

## Remove Duplicate Rows and Add New Column

In [6]:
dataset = dataset.unique()
dataset = dataset.with_columns(ps.col('PublishTime').dt.strftime('%Y-%m').alias('PublishYearAndMonth'))

## Convert Non-numerical Data to Numbers

In [7]:
dataset['County'].value_counts(sort=True)

County,counts
str,u32
"""新北市""",193702
"""高雄市""",183183
"""臺北市""",112730
"""桃園市""",92530
"""臺中市""",76110
"""臺南市""",69482
"""雲林縣""",61066
"""彰化縣""",53641
"""屏東縣""",53619


In [8]:
county_type: list = dataset['County'].value_counts(sort=True)['County'].to_list()
county_map: dict = dict(zip(county_type, [i for i in range(len(county_type))]))

In [9]:
dataset = dataset.with_columns(
	ps.col('County').map_dict(county_map, default='unknown').alias('County').cast(ps.Int32, strict=False))

In [10]:
dataset['Pollutant'].value_counts(sort=True)

Pollutant,counts
str,u32
null,758757
"""細懸浮微粒""",394179
"""臭氧八小時""",46944
"""二氧化氮""",9469
"""懸浮微粒""",8893
"""二氧化硫""",128
"""臭氧""",9
"""一氧化碳""",2


In [11]:
pollutant_type: list = dataset['Pollutant'].value_counts(sort=True)['Pollutant'].to_list()
pollutant_type.remove(None)
pollutant_map: dict = dict(zip(pollutant_type, [i for i in range(len(pollutant_type))]))

In [12]:
dataset = dataset.with_columns(
	ps.col('Pollutant').map_dict(pollutant_map, default='unknown').alias('Pollutant').cast(ps.Int32, strict=False))

In [13]:
dataset['Status'].value_counts(sort=True)

Status,counts
str,u32
"""良好""",755203
"""普通""",385681
"""對敏感族群不健康""",64309
"""對所有族群不健康""",9610
null,3554
"""非常不健康""",22
"""危害""",2


In [14]:
status_type: list = dataset['Status'].value_counts(sort=True)['Status'].to_list()
status_type.remove(None)
status_map: dict = dict(zip(status_type, [i for i in range(len(status_type))]))

In [15]:
dataset = dataset.with_columns(
	ps.col('Status').map_dict(status_map, default='unknown').alias('Status').cast(ps.Int32, strict=False))

## Fill Missing Values

In [16]:
dataset.null_count()

SiteName,County,AQI,Pollutant,Status,SO2,CO,CO_8hr,O3,O3_8hr,PM10,PM10_AVG,PM2.5,PM2.5_AVG,NO2,NOx,NO,WindSpeed,WindDirec,PublishTime,SO2_AVG,Longitude,Latitude,SiteId,PublishYearAndMonth
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,4500,758757,3554,26661,28101,17624,37485,22618,27914,16777,24383,13266,31769,31942,31897,268991,268863,0,11399,0,0,0,0


In [17]:
dataset: pd.DataFrame = dataset.to_pandas(use_pyarrow_extension_array=True)

In [18]:
processed_columns: list = ['AQI', 'Pollutant', 'Status', 'WindDirec']
for column in processed_columns:
	dataset[column] = dataset.groupby(['County', 'PublishYearAndMonth'])[column] \
		.transform(lambda group: group.fillna(group.median()) if not group.isnull().all()
	else group.fillna(dataset[column].median()))

In [19]:
processed_columns: list = ['SO2', 'SO2_AVG', 'CO', 'CO_8hr', 'O3', 'O3_8hr', 'PM10', 'PM10_AVG', 'PM2.5', 'PM2.5_AVG', 'NO2', 'NOx', 'NO', 'WindSpeed']
for column in processed_columns:
	dataset[column] = dataset.groupby(['County', 'PublishYearAndMonth'])[column] \
		.transform(lambda group: group.fillna(group.mean()) if not group.isnull().all()
	else group.fillna(dataset[column].mean()))

In [20]:
dataset: ps.DataFrame = ps.from_pandas(dataset)

In [21]:
type_map.append(ps.Utf8)
type_map[1], type_map[3], type_map[4] = ps.Int32, ps.Int32, ps.Int32
for i, column in enumerate(dataset.columns):
	dataset = dataset.with_columns(ps.col(column).cast(type_map[i], strict=False))
	if type_map[i] is ps.Float32:
		dataset = dataset.with_columns(ps.col(column).round(1))

# Data Analysis

In [22]:
dataset = dataset.with_columns(ps.col('PublishTime').dt.year().cast(ps.Int32).alias('PublishYear'))
dataset = dataset.with_columns(ps.col('PublishTime').dt.month().cast(ps.Int32).alias('PublishMonth'))
dataset = dataset.with_columns(ps.col('PublishTime').dt.day().cast(ps.Int32).alias('PublishDay'))
dataset = dataset.with_columns(ps.col('PublishTime').dt.hour().cast(ps.Int32).alias('PublishHour'))

In [23]:
dataset = dataset.drop(['SiteName', 'County', 'PublishTime', 'PublishYearAndMonth', 'Longitude', 'Latitude'])
dataset = dataset.unique()
dataset = dataset[['SiteId', 'PublishYear', 'PublishMonth', 'PublishDay', 'PublishHour', 
                   'AQI', 'Pollutant', 'Status', 'SO2', 'SO2_AVG', 'CO', 'CO_8hr', 'O3', 
                   'O3_8hr', 'PM10', 'PM10_AVG', 'NO', 'NO2', 'NOx', 'WindDirec', 
                   'WindSpeed', 'PM2.5', 'PM2.5_AVG']]

In [24]:
dataset.write_csv('full_dataset_{}.csv'.format(dataset_type))

In [25]:
dataset

SiteId,PublishYear,PublishMonth,PublishDay,PublishHour,AQI,Pollutant,Status,SO2,SO2_AVG,CO,CO_8hr,O3,O3_8hr,PM10,PM10_AVG,NO,NO2,NOx,WindDirec,WindSpeed,PM2.5,PM2.5_AVG
i32,i32,i32,i32,i32,i32,i32,i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i32,f32,f32,f32
80,2022,6,1,0,15,0,0,0.9,0.0,0.1,0.1,11.4,16.4,11.0,12.0,0.6,2.1,2.8,341,0.8,0.0,2.0
56,2022,6,1,0,23,0,0,1.5,0.0,0.2,0.1,5.5,16.799999,16.0,13.0,1.3,14.9,16.200001,227,1.6,6.0,5.0
14,2022,6,1,1,27,1,0,0.0,0.0,0.3,0.4,1.9,6.5,20.0,11.0,4.2,16.700001,20.9,133,0.6,6.0,6.0
72,2022,6,1,1,15,1,0,1.4,1.0,0.2,0.2,6.1,10.2,24.0,15.0,1.4,3.8,5.2,149,0.3,5.0,2.0
24,2022,6,1,3,27,0,0,0.1,0.0,0.3,0.3,2.4,3.4,21.0,18.0,2.8,12.7,15.6,165,0.5,7.0,8.0
45,2022,6,1,6,33,0,0,1.4,1.0,0.2,0.2,5.4,5.0,23.0,25.0,1.2,8.1,9.3,132,1.4,7.0,10.0
28,2022,6,1,6,15,0,0,0.9,0.0,0.2,0.1,14.1,14.6,14.0,15.0,0.8,2.6,3.4,185,3.2,4.0,4.0
25,2022,6,1,6,21,0,0,1.8,1.0,0.3,0.2,5.8,3.2,20.0,19.0,3.0,10.3,13.4,199,0.7,7.0,6.0
62,2022,6,1,10,13,0,0,0.9,0.5,0.2,0.1,17.200001,10.9,15.0,13.0,2.7,3.0,5.7,166,1.7,3.0,3.0


In [ ]:
if dataset_type == 'train':
	sb.set(rc={'figure.figsize': (20, 20)})
	sb.heatmap(dataset.to_pandas(use_pyarrow_extension_array=True).corr(), annot=True)